### Title: Data Handling and Cleaning
Author: Tan Zhi Lun  
Contact: zhilun296@gmail.com

The code below is used to convert and parse data into a cleaner format for backtesting.

In this section we will:
1. Combine date & time column of forex 1 min OHLC data
2. Pick out data from a specified period of time (e.g. picking out 2018 data)
3. Convert timeframe of data (e.g. 1 min to 1 hr OHLC)

## 1. Combine Date & Time Column of Forex 1 Min OHLC Data

The code below demonstrates the methodology used to combine the date and time column of forex 1 minute data. The original data (1986 to 2019) is saved in a csv file, and after the data is cleaned it is saved in a csv file once again using pandas.

<img src='./Images/section1.jpg' width="600">

The code used is as follows:

In [9]:
import pandas as pd
import datetime

# Choose pairs to clean
pairs = ("USDCAD", "AUDUSD", "GBPUSD", "USDJPY", "USDCHF")

# In order to define rule for parsing dates
def dateparse(d,t):
    dt = d + " " + t
    return datetime.datetime.strptime(dt, '%Y.%m.%d %H:%M')

# To iterate for each pair, as well as clean up index column
for i in pairs:
    filename = i + "_1_MT4.csv"
    df = pd.read_csv(filename, parse_dates={'datetime':['Date', 'Time']}, date_parser = dateparse)
    df.index = df['datetime']
    del df['datetime']
    del df['Number']
    # outname = i + "_out.csv"
    # df.to_csv(outname)
    
    #Demonstration of handling using USDCAD
    if i == "USDCAD":
        df2 = pd.read_csv(filename)
        del df2['Number']
        print("Original ",i)
        print(df2.head())
        print(" ")
        print("Cleaned ",i)
        print(df.head())

Original  USDCAD
         Date  Time    Open    High     Low   Close
0  1986.12.01  1:00  1.3806  1.3806  1.3806  1.3806
1  1986.12.01  1:01  1.3806  1.3806  1.3806  1.3806
2  1986.12.01  1:02  1.3806  1.3806  1.3806  1.3806
3  1986.12.01  1:03  1.3806  1.3806  1.3806  1.3806
4  1986.12.01  1:04  1.3806  1.3806  1.3806  1.3806
 
Cleaned  USDCAD
                       Open    High     Low   Close
datetime                                           
1986-12-01 01:00:00  1.3806  1.3806  1.3806  1.3806
1986-12-01 01:01:00  1.3806  1.3806  1.3806  1.3806
1986-12-01 01:02:00  1.3806  1.3806  1.3806  1.3806
1986-12-01 01:03:00  1.3806  1.3806  1.3806  1.3806
1986-12-01 01:04:00  1.3806  1.3806  1.3806  1.3806


## 2. Pick Out Data From Specified Period of Time

In practice, data from too long ago is often unusable due to reasons such as regime shifts.

Thus, the code below demonstrates the methodology used to pick out any period of time from the dataframe. For the example given the period chosen starts at 2nd Jan 2017 0830 hrs and ends at 31 Jan 2017 2359 hrs.

<img src='./Images/section2.jpg' width="600">

In [20]:
import pandas as pd
from datetime import datetime

# Define pairs that are to be cleaned
pairs = ("USDCAD", "AUDUSD", "GBPUSD", "USDJPY", "USDCHF")

# Choose starting and end date
year = 2017
startdate = datetime(year,1,2,8,30)
enddate = datetime(year,1,31,23,59)

# Load pairs
for i in pairs:
    filename = i + '_out.csv'
    outfile = i + '_out_2017.csv'
    
    # Set index column to datetime and parse by using to datetime, otherwise str wont be recognized for < & >
    df = pd.read_csv(filename, index_col='datetime')
    df.index = pd.to_datetime(df.index)
    del df['Number']
    
    # sdf for ranging purposes, in case certain years dont start on 1st Jan etc
    sdf = (df.index>=startdate) & (df.index<=enddate)
    
    # loc by mask
    df2 = df.loc[sdf]
    
    # Output to csv file
    # df2.to_csv(outfile, index = 'datetime')
    
    print(i)
    print(df2.head())
    print(" ")

USDCAD
                        Open     High      Low    Close
datetime                                               
2017-01-02 08:30:00  1.34416  1.34421  1.34412  1.34418
2017-01-02 08:31:00  1.34430  1.34433  1.34416  1.34428
2017-01-02 08:32:00  1.34418  1.34426  1.34415  1.34426
2017-01-02 08:33:00  1.34426  1.34429  1.34414  1.34414
2017-01-02 08:34:00  1.34414  1.34418  1.34412  1.34415
 
AUDUSD
                        Open     High      Low    Close
datetime                                               
2017-01-02 08:30:00  0.72069  0.72095  0.72028  0.72028
2017-01-02 08:31:00  0.72037  0.72081  0.72037  0.72053
2017-01-02 08:32:00  0.72053  0.72110  0.72052  0.72089
2017-01-02 08:33:00  0.72089  0.72090  0.72076  0.72090
2017-01-02 08:34:00  0.72089  0.72089  0.72089  0.72089
 
GBPUSD
                        Open     High      Low    Close
datetime                                               
2017-01-02 08:30:00  1.23431  1.23440  1.23431  1.23440
2017-01-02 08:31:00  1.

## 3. Convert Timeframe of Data

In practice, different timeframes can be used for signal generation, e.g. higher timeframe data would better show the big picture trend.

The code below demonstrates the methodology used combine the lower timeframe OHLC data. For the example given it combines 1 minute data into 5 minute data.

<img src='./Images/section3.jpg' width="600">

In [11]:
import pandas as pd
import math

minutes = 5
filename = "eurusd_out_2018.csv"
outname = "eurusd_out_2018_5mins.csv"


df = pd.read_csv(filename,index_col= False) 
print(df.head())

        datetime     Open     High      Low    Close
0  2/1/2018 0:00  1.20126  1.20126  1.20097  1.20103
1  2/1/2018 0:01  1.20103  1.20105  1.19998  1.20048
2  2/1/2018 0:02  1.20048  1.20057  1.19998  1.20043
3  2/1/2018 0:03  1.20046  1.20076  1.20035  1.20062
4  2/1/2018 0:04  1.20061  1.20070  1.20051  1.20065


In [9]:
df5=df.copy()
df6=df.copy()

for i in df.index:
    n = math.floor(i/minutes) * minutes
    n2 = (n + minutes -1)
    df5.loc[i,'High'] = df.loc[n:n2,'High'].max()

for i in df.index:
    n = math.floor(i/minutes) * minutes
    n2 = (n + minutes -1)
    df6.loc[i,'Low'] = df.loc[n:n2,'Low'].min()

df5 = df5.iloc[::minutes]['High']
df6 = df6.iloc[::minutes]['Low']
print(df5)
print(df6)

0     1.20126
5     1.20100
10    1.20144
15    1.20153
20    1.20133
25    1.20138
Name: High, dtype: float64
0     1.19998
5     1.19970
10    1.20077
15    1.20128
20    1.20090
25    1.20106
Name: Low, dtype: float64


In [12]:
# Slicing the dataframes such that they can be combined later on
df2 = df.iloc[::minutes]
df3 = df.iloc[(minutes-1)::minutes]['Close']
df2 = df2.drop(['High'], axis = 1)
df2 = df2.drop(['Low'], axis = 1)
df2 = df2.drop(['Close'], axis = 1)

# Reset the index such that the dataframes can be merged accordingly
df2 = df2.reset_index(drop=True)
df3 = df3.reset_index(drop=True)
df5 = df5.reset_index(drop=True)
df6 = df6.reset_index(drop=True)
df4 = pd.merge(df2, df5, left_index = True, right_index= True)
df4 = pd.merge(df4, df6, left_index = True, right_index= True)
df4 = pd.merge(df4, df3, left_index = True, right_index= True)

# Rearranging
cols = df.columns.tolist()
cols = cols[:4] + [cols[-2]] + [cols[-1]]
df4 = df4[cols]

# Resetting the index
df4.index = df4['datetime']
del df4['datetime']
df4.to_csv(outname,index='datetime')

print(df4.head())

                  Open     High      Low      Low    Close
datetime                                                  
2/1/2018 0:00  1.20126  1.20126  1.19998  1.19998  1.20065
2/1/2018 0:05  1.20067  1.20100  1.19970  1.19970  1.20086
2/1/2018 0:10  1.20085  1.20144  1.20077  1.20077  1.20144
2/1/2018 0:15  1.20140  1.20153  1.20128  1.20128  1.20134
2/1/2018 0:20  1.20131  1.20133  1.20090  1.20090  1.20113
